# Paper-Ready Experiments

This notebook completes critical experiments needed before paper submission:

## Priority 1 (Must-Do)
1. **D8_muq_stats completion**: Run missing folds 0, 1, 2 (fold 3 exists with R2=0.560)
2. **PSyllabus cross-validation**: Validate model on external difficulty dataset

## Priority 2 (Should-Do)
3. **Performer-fold evaluation**: Test generalization to unseen performers

## Priority 3 (Nice-to-Have)
4. **Multi-model performer-fold comparison**: Compare MuQ, MERT, symbolic on performer splits
5. **Soundfont augmentation**: Re-render MIDI with multiple Pianoteq presets for data augmentation

## Requirements
- Compute: A100 (80GB VRAM)
- rclone configured with `gdrive:` remote
- MuQ embeddings (will be extracted if not cached)
- For soundfont augmentation: Pianoteq CLI installed

In [ ]:
# Cell 1: CUDA setup (must be before any CUDA operations)
import os
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    raise RuntimeError("GPU required")

In [ ]:
# Cell 2: Install rclone
!curl -fsSL https://rclone.org/install.sh | sudo bash 2>&1 | grep -E "(successfully|already)" || echo "rclone installed"

In [ ]:
# Cell 3: Install dependencies and clone repo
!pip install transformers librosa soundfile pytorch_lightning nnAudio scipy scikit-learn muq requests tqdm --quiet

import os
REPO_DIR = '/tmp/crescendai'
if os.path.exists(REPO_DIR):
    !cd {REPO_DIR} && git pull origin main
else:
    !git clone https://github.com/jai-dhiman/crescendai.git {REPO_DIR}

print(f"Repo: {REPO_DIR}")

In [ ]:
# Cell 4: Imports
import sys
sys.path.insert(0, f'{REPO_DIR}/model/src')

import json
import subprocess
import warnings
from pathlib import Path
from typing import Dict, List, Tuple, Any, Optional

import numpy as np
import torch
import pytorch_lightning as pl
from scipy import stats
from sklearn.metrics import r2_score
from tqdm.auto import tqdm

from audio_experiments import PERCEPIANO_DIMENSIONS, DIMENSION_CATEGORIES, BASE_CONFIG, SEED
from audio_experiments.extractors import extract_muq_embeddings
from audio_experiments.models import MuQStatsModel
from audio_experiments.training import (
    run_4fold_mert_experiment,
    should_run_experiment,
    sync_experiment_to_gdrive,
    get_completed_experiments,
    print_experiment_status,
    bootstrap_r2_extended,
)
from audio_experiments.training.sync import numpy_serializer
from audio_experiments.training.metrics import compute_comprehensive_metrics

warnings.filterwarnings('ignore')
torch.set_float32_matmul_precision('medium')
pl.seed_everything(SEED, workers=True)

print(f"PyTorch: {torch.__version__}")
print(f"Imports: OK")

In [ ]:
# Cell 5: Path configuration
DATA_ROOT = Path('/tmp/paper_ready_experiments')
AUDIO_DIR = DATA_ROOT / 'audio'
LABEL_DIR = DATA_ROOT / 'labels'
MUQ_CACHE_ROOT = DATA_ROOT / 'muq_cache'
CHECKPOINT_ROOT = DATA_ROOT / 'checkpoints'
RESULTS_DIR = DATA_ROOT / 'results'
LOG_DIR = DATA_ROOT / 'logs'
FIGURES_DIR = RESULTS_DIR / 'figures'

# Cross-dataset directories
PSYLLABUS_DIR = DATA_ROOT / 'psyllabus'

# GDrive paths
GDRIVE_AUDIO = 'gdrive:crescendai_data/audio_baseline/percepiano_rendered'
GDRIVE_LABELS = 'gdrive:crescendai_data/percepiano_labels'
GDRIVE_FOLDS = 'gdrive:crescendai_data/percepiano_fold_assignments.json'
GDRIVE_MUQ_CACHE = 'gdrive:crescendai_data/audio_baseline/muq_embeddings'
# D8 results are in audio_phase2 (original experiment location)
GDRIVE_D8_RESULTS = 'gdrive:crescendai_data/checkpoints/audio_phase2'
# New experiments go to paper_ready directory
GDRIVE_RESULTS = 'gdrive:crescendai_data/checkpoints/paper_ready_experiments'

for d in [AUDIO_DIR, LABEL_DIR, MUQ_CACHE_ROOT, CHECKPOINT_ROOT,
          RESULTS_DIR, LOG_DIR, FIGURES_DIR, PSYLLABUS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

def run_rclone(cmd, desc=""):
    if desc:
        print(f"{desc}...")
    result = subprocess.run(cmd, capture_output=True, text=True)
    if result.returncode != 0:
        raise RuntimeError(f"rclone failed: {desc}\nCommand: {' '.join(cmd)}\nStderr: {result.stderr}")
    return result

# Check rclone
result = subprocess.run(['rclone', 'listremotes'], capture_output=True, text=True)
if 'gdrive:' not in result.stdout:
    raise RuntimeError("rclone 'gdrive' not configured")

print(f"Data root: {DATA_ROOT}")
print(f"GDrive D8 results: {GDRIVE_D8_RESULTS}")
print(f"GDrive new results: {GDRIVE_RESULTS}")

In [ ]:
# Cell 6: Download data
run_rclone(['rclone', 'copy', GDRIVE_AUDIO, str(AUDIO_DIR), '--progress'], "Downloading audio")
run_rclone(['rclone', 'copy', GDRIVE_LABELS, str(LABEL_DIR)], "Downloading labels")

FOLD_FILE = DATA_ROOT / 'folds.json'
run_rclone(['rclone', 'copyto', GDRIVE_FOLDS, str(FOLD_FILE)], "Downloading folds")

# Load labels and folds
LABEL_FILE = LABEL_DIR / 'label_2round_mean_reg_19_with0_rm_highstd0.json'
with open(LABEL_FILE) as f:
    LABELS = json.load(f)
with open(FOLD_FILE) as f:
    FOLD_ASSIGNMENTS = json.load(f)

# Create key->fold_id mapping
FOLD_BY_KEY = {}
for fold_id in range(4):
    for key in FOLD_ASSIGNMENTS.get(f"fold_{fold_id}", []):
        FOLD_BY_KEY[key] = fold_id

ALL_KEYS = sorted(FOLD_BY_KEY.keys())
print(f"Samples per fold: {[len(FOLD_ASSIGNMENTS.get(f'fold_{i}', [])) for i in range(4)]}")
print(f"Total samples: {len(ALL_KEYS)}")
print(f"Audio files: {len(list(AUDIO_DIR.glob('*.wav')))}")

In [ ]:
# Cell 7: Initialize results tracking
ALL_RESULTS = {}

# Get completed experiments from GDrive (both D8 location and new location)
print("Checking GDrive for completed experiments...")
D8_COMPLETED = get_completed_experiments(GDRIVE_D8_RESULTS)
NEW_COMPLETED = get_completed_experiments(GDRIVE_RESULTS)
COMPLETED_CACHE = {**D8_COMPLETED, **NEW_COMPLETED}

print(f"Found {len(D8_COMPLETED)} experiments in audio_phase2")
print(f"Found {len(NEW_COMPLETED)} experiments in paper_ready_experiments")

# Define experiment IDs for this notebook
EXPERIMENT_IDS = [
    # Part 1: Complete D8_muq_stats (Priority 1)
    'D8_muq_stats',  # Complete folds 0, 1, 2
    # Part 2: PSyllabus Cross-Validation (Priority 1)
    'X3_psyllabus_difficulty',
    # Part 3: Performer-Fold Evaluation (Priority 2)
    'P1_performer_fold_muq',
    # Part 5: Multi-Model Performer-Fold (Priority 3)
    'P2_performer_fold_mert',
    # Part 6: Soundfont Augmentation (Priority 3)
    'S1_soundfont_augmented',
]

print_experiment_status(EXPERIMENT_IDS, COMPLETED_CACHE)

---
## Part 1: Complete D8_muq_stats Cross-Validation

The D8_muq_stats experiment (MuQ + stats pooling using last_hidden_state) currently only has fold 3 completed with R2 = 0.560.

**Critical Issue:** This single fold cannot be used as a headline result. We need all 4 folds for proper cross-validation.

**Expected outcome:** With 4-fold average, likely R2 ~ 0.52-0.55 (based on fold variance in M1c_muq_L9-12).

In [ ]:
# Cell 9: Check existing D8_muq_stats folds
D8_CKPT_PATH = 'gdrive:crescendai_data/checkpoints/audio_phase2/checkpoints/D8_muq_stats'

# Check which folds exist
result = subprocess.run(['rclone', 'lsf', D8_CKPT_PATH], capture_output=True, text=True)
existing_files = set(result.stdout.strip().split('\n')) if result.returncode == 0 else set()

existing_folds = []
missing_folds = []

for fold in range(4):
    if f'fold{fold}_best.ckpt' in existing_files:
        existing_folds.append(fold)
    else:
        missing_folds.append(fold)

print(f"D8_muq_stats status:")
print(f"  Existing folds: {existing_folds}")
print(f"  Missing folds: {missing_folds}")

if not missing_folds:
    print("\nAll folds complete! No training needed.")
else:
    print(f"\nNeed to train folds: {missing_folds}")

In [ ]:
# Cell 10: Download existing D8 checkpoints
if existing_folds:
    D8_LOCAL_CKPT = CHECKPOINT_ROOT / 'D8_muq_stats'
    D8_LOCAL_CKPT.mkdir(parents=True, exist_ok=True)
    
    print(f"Downloading existing D8 checkpoints...")
    run_rclone(['rclone', 'copy', D8_CKPT_PATH, str(D8_LOCAL_CKPT)], "Downloading D8 checkpoints")
    
    # Verify downloads
    downloaded = list(D8_LOCAL_CKPT.glob('*.ckpt'))
    print(f"Downloaded {len(downloaded)} checkpoint files")

In [ ]:
# Cell 11: Download or extract MuQ embeddings (last_hidden_state)
# D8 uses last_hidden_state, not layer ranges
MUQ_CACHE_DIR = MUQ_CACHE_ROOT / 'last_hidden_state'
MUQ_CACHE_DIR.mkdir(parents=True, exist_ok=True)

# Try to download cached embeddings first
GDRIVE_MUQ_LHS = 'gdrive:crescendai_data/audio_baseline/muq_embeddings/last_hidden_state'
result = subprocess.run(['rclone', 'lsf', GDRIVE_MUQ_LHS], capture_output=True, text=True)

if result.returncode == 0 and result.stdout.strip():
    print("Downloading cached MuQ embeddings (last_hidden_state)...")
    run_rclone(['rclone', 'copy', GDRIVE_MUQ_LHS, str(MUQ_CACHE_DIR), '--progress'], 
               "Downloading MuQ embeddings")
else:
    print("No cached MuQ embeddings found. Will extract from audio.")

# Check what we have
cached_keys = {p.stem for p in MUQ_CACHE_DIR.glob('*.pt')}
missing_keys = [k for k in ALL_KEYS if k not in cached_keys]
print(f"Cached: {len(cached_keys)}, Missing: {len(missing_keys)}")

# Extract missing embeddings
if missing_keys:
    print(f"\nExtracting {len(missing_keys)} missing MuQ embeddings...")
    # layer_start=None, layer_end=None -> uses last_hidden_state
    extract_muq_embeddings(AUDIO_DIR, MUQ_CACHE_DIR, missing_keys, layer_start=None, layer_end=None)
    
    # Upload newly extracted embeddings to GDrive
    print("\nUploading MuQ embeddings to GDrive...")
    run_rclone(['rclone', 'copy', str(MUQ_CACHE_DIR), GDRIVE_MUQ_LHS], 
               "Uploading MuQ embeddings")

In [ ]:
# Cell 12: D8_muq_stats model configuration
MUQ_STATS_CONFIG = {
    **BASE_CONFIG,
    'input_dim': 1024,
    'hidden_dim': 512,
    'dropout': 0.2,
    'learning_rate': 1e-4,
    'weight_decay': 1e-5,
    'pooling_stats': 'mean_std',  # 2x input dim
}

def make_muq_stats_model(cfg):
    return MuQStatsModel(
        input_dim=cfg['input_dim'],
        hidden_dim=cfg['hidden_dim'],
        dropout=cfg['dropout'],
        learning_rate=cfg['learning_rate'],
        weight_decay=cfg['weight_decay'],
        pooling_stats=cfg['pooling_stats'],
        max_epochs=cfg['max_epochs'],
    )

print("D8_muq_stats configuration:")
print(f"  Input dim: {MUQ_STATS_CONFIG['input_dim']}")
print(f"  Hidden dim: {MUQ_STATS_CONFIG['hidden_dim']}")
print(f"  Pooling: {MUQ_STATS_CONFIG['pooling_stats']}")
print(f"  Learning rate: {MUQ_STATS_CONFIG['learning_rate']}")
print(f"  Max epochs: {MUQ_STATS_CONFIG['max_epochs']}")

In [ ]:
# Cell 13: Train missing D8_muq_stats folds
from torch.utils.data import DataLoader
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger
from audio_experiments.data import MERTDataset, mert_collate_fn

exp_id = 'D8_muq_stats'
exp_checkpoint_dir = CHECKPOINT_ROOT / exp_id
exp_checkpoint_dir.mkdir(parents=True, exist_ok=True)

if missing_folds:
    print(f"\n{'='*70}")
    print(f"EXPERIMENT: {exp_id} (completing folds {missing_folds})")
    print(f"Description: MuQ with stats pooling (mean+std)")
    print(f"{'='*70}")
    
    fold_results = {}
    
    # Load existing fold results if available
    existing_results_file = RESULTS_DIR / f'{exp_id}.json'
    if existing_results_file.exists():
        with open(existing_results_file) as f:
            existing_data = json.load(f)
            if 'fold_results' in existing_data:
                fold_results = {int(k): v for k, v in existing_data['fold_results'].items()}
                print(f"Loaded existing fold results: {fold_results}")
    
    # Train only missing folds
    for fold in missing_folds:
        ckpt_path = exp_checkpoint_dir / f"fold{fold}_best.ckpt"
        
        # Create datasets
        train_ds = MERTDataset(
            MUQ_CACHE_DIR, LABELS, FOLD_ASSIGNMENTS, fold, "train", MUQ_STATS_CONFIG["max_frames"]
        )
        val_ds = MERTDataset(
            MUQ_CACHE_DIR, LABELS, FOLD_ASSIGNMENTS, fold, "val", MUQ_STATS_CONFIG["max_frames"]
        )
        
        print(f"\nFold {fold}: Training ({len(train_ds)} train, {len(val_ds)} val)")
        
        train_dl = DataLoader(
            train_ds,
            batch_size=MUQ_STATS_CONFIG["batch_size"],
            shuffle=True,
            collate_fn=mert_collate_fn,
            num_workers=MUQ_STATS_CONFIG["num_workers"],
            pin_memory=True,
        )
        val_dl = DataLoader(
            val_ds,
            batch_size=MUQ_STATS_CONFIG["batch_size"],
            shuffle=False,
            collate_fn=mert_collate_fn,
            num_workers=MUQ_STATS_CONFIG["num_workers"],
            pin_memory=True,
        )
        
        model = make_muq_stats_model(MUQ_STATS_CONFIG)
        
        callbacks = [
            ModelCheckpoint(
                dirpath=exp_checkpoint_dir,
                filename=f"fold{fold}_best",
                monitor="val_r2",
                mode="max",
                save_top_k=1,
            ),
            EarlyStopping(
                monitor="val_r2", mode="max", patience=MUQ_STATS_CONFIG["patience"], verbose=True
            ),
        ]
        
        trainer = pl.Trainer(
            max_epochs=MUQ_STATS_CONFIG["max_epochs"],
            callbacks=callbacks,
            logger=CSVLogger(save_dir=LOG_DIR, name=exp_id, version=f"fold{fold}"),
            accelerator="auto",
            devices=1,
            gradient_clip_val=MUQ_STATS_CONFIG["gradient_clip_val"],
            enable_progress_bar=True,
            deterministic=True,
            log_every_n_steps=10,
        )
        
        trainer.fit(model, train_dl, val_dl)
        fold_results[fold] = float(callbacks[0].best_model_score or 0)
        print(f"Fold {fold} complete: val_r2 = {fold_results[fold]:.4f}")
        
        # Upload checkpoint immediately
        print(f"Uploading fold {fold} checkpoint...")
        run_rclone(['rclone', 'copy', str(exp_checkpoint_dir), D8_CKPT_PATH],
                   f"Uploading fold {fold}")
        
        del model, trainer
        torch.cuda.empty_cache()
    
    print(f"\nAll missing folds trained!")
    print(f"Fold results: {fold_results}")
else:
    print(f"\n{exp_id}: All folds already complete, skipping training.")

In [ ]:
# Cell 14: Compute comprehensive D8_muq_stats metrics
exp_id = 'D8_muq_stats'
exp_checkpoint_dir = CHECKPOINT_ROOT / exp_id

print(f"\n{'='*70}")
print(f"COMPUTING COMPREHENSIVE METRICS: {exp_id}")
print(f"{'='*70}")

# Collect predictions from all folds
all_preds = []
all_labels = []
fold_r2_scores = {}

for fold in range(4):
    ckpt_path = exp_checkpoint_dir / f"fold{fold}_best.ckpt"
    if not ckpt_path.exists():
        raise FileNotFoundError(f"Missing checkpoint: {ckpt_path}")
    
    # Load model
    model = MuQStatsModel.load_from_checkpoint(ckpt_path)
    model = model.to('cuda').eval()
    
    # Get validation data for this fold
    val_ds = MERTDataset(
        MUQ_CACHE_DIR, LABELS, FOLD_ASSIGNMENTS, fold, "val", MUQ_STATS_CONFIG["max_frames"]
    )
    val_dl = DataLoader(
        val_ds,
        batch_size=MUQ_STATS_CONFIG["batch_size"],
        shuffle=False,
        collate_fn=mert_collate_fn,
        num_workers=MUQ_STATS_CONFIG["num_workers"],
        pin_memory=True,
    )
    
    fold_preds = []
    fold_labels = []
    
    with torch.no_grad():
        for batch in val_dl:
            pred = model(
                batch["embeddings"].cuda(),
                batch["attention_mask"].cuda(),
                batch.get("lengths"),
            )
            fold_preds.append(pred.cpu().numpy())
            fold_labels.append(batch["labels"].numpy())
    
    fold_preds = np.vstack(fold_preds)
    fold_labels = np.vstack(fold_labels)
    
    # Compute fold-level R2
    fold_r2 = r2_score(fold_labels, fold_preds)
    fold_r2_scores[fold] = fold_r2
    print(f"Fold {fold}: R2 = {fold_r2:.4f} ({len(val_ds)} samples)")
    
    all_preds.append(fold_preds)
    all_labels.append(fold_labels)
    
    del model
    torch.cuda.empty_cache()

# Aggregate all predictions
all_preds = np.vstack(all_preds)
all_labels = np.vstack(all_labels)

# Compute comprehensive metrics (signature: all_preds, all_labels)
metrics = compute_comprehensive_metrics(all_preds, all_labels)

# Compute proper 4-fold statistics
fold_r2_values = list(fold_r2_scores.values())
avg_r2 = np.mean(fold_r2_values)
std_r2 = np.std(fold_r2_values)

# Bootstrap CI (signature: y_true, y_pred - so labels first!)
bootstrap_results = bootstrap_r2_extended(all_labels, all_preds, n_bootstrap=1000)

# Extract CI from the nested structure
ci_lower = bootstrap_results['overall']['ci_lower']
ci_upper = bootstrap_results['overall']['ci_upper']

print(f"\n" + "="*50)
print(f"D8_muq_stats FINAL RESULTS (4-fold CV):")
print(f"="*50)
print(f"Per-fold R2: {[f'{v:.4f}' for v in fold_r2_values]}")
print(f"Average R2: {avg_r2:.4f} +/- {std_r2:.4f}")
print(f"Overall R2 (pooled): {metrics['overall_r2']:.4f}")
print(f"Bootstrap 95% CI: [{ci_lower:.4f}, {ci_upper:.4f}]")
print(f"Overall MAE: {metrics['overall_mae']:.4f}")
print(f"Dispersion ratio: {metrics['dispersion_ratio']:.4f}")

In [ ]:
# Cell 15: Save D8_muq_stats final results
exp_id = 'D8_muq_stats'

d8_results = {
    "experiment_id": exp_id,
    "description": "MuQ with stats pooling (mean+std) - COMPLETE 4-fold CV",
    "config": MUQ_STATS_CONFIG,
    "summary": {
        "avg_r2": float(avg_r2),
        "std_r2": float(std_r2),
        "r2_ci_95": [float(ci_lower), float(ci_upper)],
        "overall_r2": float(metrics['overall_r2']),
        "overall_mae": float(metrics['overall_mae']),
        "dispersion_ratio": float(metrics['dispersion_ratio']),
    },
    "fold_results": {str(k): float(v) for k, v in fold_r2_scores.items()},
    "per_dimension": metrics['per_dimension'],
    "note": "Completed with all 4 folds. Previous result (R2=0.560) was fold 3 only.",
}

# Save locally
with open(RESULTS_DIR / f'{exp_id}.json', 'w') as f:
    json.dump(d8_results, f, indent=2, default=numpy_serializer)

ALL_RESULTS[exp_id] = d8_results

# Sync to GDrive (update the original location)
print(f"\nSyncing {exp_id} to GDrive...")
run_rclone(['rclone', 'copyto', 
            str(RESULTS_DIR / f'{exp_id}.json'),
            f'{GDRIVE_D8_RESULTS}/{exp_id}.json'],
           f"Uploading {exp_id} results")

print(f"\n{exp_id} COMPLETE: avg_r2={avg_r2:.4f}, CI=[{ci_lower:.4f}, {ci_upper:.4f}]")

---
## Part 2: PSyllabus Cross-Dataset Validation

PSyllabus is an external dataset with piano pieces labeled by difficulty level (1-11).

**Goal:** Validate that our model captures musically meaningful features by correlating predictions with difficulty levels.

**Hypothesis:** Higher difficulty pieces should correlate with certain dimensions (e.g., tempo, articulation complexity).

In [ ]:
# Cell 17: PSyllabus Dataset Setup
import time
import random
import requests

# Install yt-dlp if not available
try:
    import yt_dlp
except ImportError:
    print("Installing yt-dlp...")
    subprocess.run(['pip', 'install', '-q', 'yt-dlp'], check=True)
    import yt_dlp

PSYLLABUS_METADATA_URL = 'https://zenodo.org/records/14794592/files/new_clean_data.json?download=1'
PSYLLABUS_METADATA_FILE = PSYLLABUS_DIR / 'new_clean_data.json'
PSYLLABUS_AUDIO_DIR = PSYLLABUS_DIR / 'audio'
PSYLLABUS_CHECKPOINT_FILE = PSYLLABUS_DIR / 'download_checkpoint.json'

PSYLLABUS_AUDIO_DIR.mkdir(parents=True, exist_ok=True)

def download_with_progress(url: str, dest: Path, desc: str):
    """Download file with progress bar."""
    response = requests.get(url, stream=True)
    response.raise_for_status()
    total = int(response.headers.get('content-length', 0))
    
    with open(dest, 'wb') as f:
        with tqdm(total=total, unit='B', unit_scale=True, desc=desc) as pbar:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
                pbar.update(len(chunk))

# Step 1: Download metadata from Zenodo
if not PSYLLABUS_METADATA_FILE.exists():
    print("Downloading PSyllabus metadata from Zenodo...")
    download_with_progress(PSYLLABUS_METADATA_URL, PSYLLABUS_METADATA_FILE, "PSyllabus metadata")
else:
    print(f"PSyllabus metadata exists: {PSYLLABUS_METADATA_FILE}")

# Step 2: Load and parse metadata
with open(PSYLLABUS_METADATA_FILE) as f:
    psyllabus_raw = json.load(f)

# Debug: show raw data structure
print(f"\nRaw data type: {type(psyllabus_raw)}")

# Parse entries - PSyllabus data is a DICT with piece names as keys
psyllabus_entries = []
if isinstance(psyllabus_raw, list):
    print("Data is a list - using directly")
    psyllabus_entries = psyllabus_raw
elif isinstance(psyllabus_raw, dict):
    print(f"Data is a dict with {len(psyllabus_raw)} keys")
    if 'data' in psyllabus_raw:
        print("Found 'data' key - using that")
        psyllabus_entries = psyllabus_raw['data']
    else:
        # Convert dict to list, adding the key as 'id'
        print("Converting dict to list of entries...")
        psyllabus_entries = [{'id': k, **v} for k, v in psyllabus_raw.items()]

print(f"\nPSyllabus entries loaded: {len(psyllabus_entries)}")
if psyllabus_entries:
    first_entry = psyllabus_entries[0]
    print(f"Sample entry keys: {list(first_entry.keys())}")
    print(f"\nFirst entry sample values:")
    print(f"  youtube_link: {first_entry.get('youtube_link', 'MISSING')}")
    print(f"  ps_rating: {first_entry.get('ps_rating', 'MISSING')}")
    print(f"  ps: {first_entry.get('ps', 'MISSING')}")

In [ ]:
# Cell 18: Parse PSyllabus entries

# First, let's debug by examining actual field values
print("Debugging PSyllabus data structure...")
if psyllabus_entries:
    sample = psyllabus_entries[0]
    print(f"\nSample entry (first record):")
    for key, value in sample.items():
        print(f"  {key}: {repr(value)[:100]}")
    
    # Check a few more entries
    print(f"\nChecking youtube_link and difficulty fields across first 5 entries:")
    for i, entry in enumerate(psyllabus_entries[:5]):
        yt_link = entry.get('youtube_link', 'MISSING')
        ps_rating = entry.get('ps_rating', 'MISSING')
        syllabus = entry.get('syllabus', 'MISSING')
        ps = entry.get('ps', 'MISSING')
        print(f"  [{i}] youtube_link={repr(yt_link)[:50]}, ps_rating={repr(ps_rating)}, syllabus={repr(syllabus)}, ps={repr(ps)}")

def extract_youtube_id(entry):
    """Extract YouTube video ID from various possible fields."""
    # Check youtube_link first (the actual field in PSyllabus)
    for field in ['youtube_link', 'youtube_id', 'video_id', 'url', 'youtube_url', 'link']:
        if field in entry:
            val = entry[field]
            if val:
                val = str(val)
                # Handle full YouTube URLs
                if 'youtube.com' in val or 'youtu.be' in val:
                    if 'v=' in val:
                        return val.split('v=')[1].split('&')[0].split('?')[0]
                    elif 'youtu.be/' in val:
                        return val.split('youtu.be/')[1].split('?')[0].split('&')[0]
                    # Handle embed URLs like youtube.com/embed/VIDEO_ID
                    elif '/embed/' in val:
                        return val.split('/embed/')[1].split('?')[0].split('&')[0]
                # Check if it's a raw 11-character video ID
                elif len(val) == 11:
                    return val
    return None

def extract_difficulty(entry):
    """Extract difficulty level from various possible fields."""
    # PSyllabus uses 'ps_rating', 'syllabus', or 'ps' for difficulty
    for field in ['ps_rating', 'ps', 'syllabus', 'difficulty', 'level', 'grade', 'difficulty_level']:
        if field in entry:
            val = entry[field]
            if val is not None and val != '':
                try:
                    # Handle string values like "1", "2", etc.
                    # Also handle float strings like "5.0"
                    diff = int(float(str(val)))
                    if 1 <= diff <= 11:
                        return diff
                except (ValueError, TypeError):
                    pass
    return None

# Parse entries
parsed_entries = []
parse_failures = {'no_youtube': 0, 'no_difficulty': 0, 'both_missing': 0}

for entry in psyllabus_entries:
    yt_id = extract_youtube_id(entry)
    difficulty = extract_difficulty(entry)
    
    if not yt_id and not difficulty:
        parse_failures['both_missing'] += 1
    elif not yt_id:
        parse_failures['no_youtube'] += 1
    elif not difficulty:
        parse_failures['no_difficulty'] += 1
    
    if yt_id and difficulty:
        parsed_entries.append({
            'youtube_id': yt_id,
            'difficulty': difficulty,
            'composer': entry.get('composer', ''),
            'title': entry.get('title', entry.get('name', '')),
        })

print(f"\n\nParsing Results:")
print(f"  Successfully parsed: {len(parsed_entries)}")
print(f"  Failed - no YouTube ID: {parse_failures['no_youtube']}")
print(f"  Failed - no difficulty: {parse_failures['no_difficulty']}")
print(f"  Failed - both missing: {parse_failures['both_missing']}")

# Show some examples if parsing worked
if parsed_entries:
    print(f"\nExample parsed entries:")
    for entry in parsed_entries[:3]:
        print(f"  {entry['composer']}: {entry['title']} (difficulty={entry['difficulty']}, yt_id={entry['youtube_id']})")
else:
    print("\nWARNING: No entries were successfully parsed!")
    print("Checking why parsing failed on first few entries...")
    for i, entry in enumerate(psyllabus_entries[:3]):
        yt_id = extract_youtube_id(entry)
        difficulty = extract_difficulty(entry)
        print(f"\n  Entry {i}:")
        print(f"    youtube_link raw value: {repr(entry.get('youtube_link', 'MISSING'))}")
        print(f"    extracted yt_id: {yt_id}")
        print(f"    ps_rating raw value: {repr(entry.get('ps_rating', 'MISSING'))}")
        print(f"    ps raw value: {repr(entry.get('ps', 'MISSING'))}")
        print(f"    syllabus raw value: {repr(entry.get('syllabus', 'MISSING'))}")
        print(f"    extracted difficulty: {difficulty}")

# Group by difficulty level
by_difficulty = {i: [] for i in range(1, 12)}
for entry in parsed_entries:
    by_difficulty[entry['difficulty']].append(entry)

print("\nDistribution by difficulty level:")
for diff, entries in sorted(by_difficulty.items()):
    print(f"  Level {diff:2d}: {len(entries):4d} entries")

In [ ]:
# Cell 19: Stratified sampling for PSyllabus
TARGET_PER_LEVEL = 30  # Reduced from 50 to speed up downloads
sampled_entries = []

random.seed(42)  # Reproducible sampling
for diff in range(1, 12):
    available = by_difficulty[diff]
    if len(available) <= TARGET_PER_LEVEL:
        sampled_entries.extend(available)
    else:
        sampled_entries.extend(random.sample(available, TARGET_PER_LEVEL))

print(f"Sampled {len(sampled_entries)} entries for download")

# Show distribution
sampled_by_diff = {}
for entry in sampled_entries:
    d = entry['difficulty']
    sampled_by_diff[d] = sampled_by_diff.get(d, 0) + 1

print("\nSampled distribution:")
for diff in range(1, 12):
    print(f"  Level {diff:2d}: {sampled_by_diff.get(diff, 0):3d} entries")

In [ ]:
# Cell 20: Download PSyllabus audio from YouTube
# Load checkpoint if exists (for resumability)
download_checkpoint = {'completed': [], 'failed': []}
if PSYLLABUS_CHECKPOINT_FILE.exists():
    with open(PSYLLABUS_CHECKPOINT_FILE) as f:
        download_checkpoint = json.load(f)
    print(f"Loaded checkpoint: {len(download_checkpoint['completed'])} completed, {len(download_checkpoint['failed'])} failed")

completed_ids = set(download_checkpoint['completed'])
failed_ids = set(download_checkpoint['failed'])

# Also check for existing audio files
existing_audio = {p.stem for p in PSYLLABUS_AUDIO_DIR.glob('*.wav')}
completed_ids.update(existing_audio)

# Filter to entries not yet attempted
to_download = [e for e in sampled_entries if e['youtube_id'] not in completed_ids and e['youtube_id'] not in failed_ids]
print(f"\nEntries to download: {len(to_download)}")

def download_youtube_audio(youtube_id: str, output_dir: Path, max_retries: int = 3) -> bool:
    """Download audio from YouTube using yt-dlp."""
    output_template = str(output_dir / f"{youtube_id}.%(ext)s")
    
    ydl_opts = {
        'format': 'bestaudio/best',
        'extractaudio': True,
        'audioformat': 'wav',
        'outtmpl': output_template,
        'quiet': True,
        'no_warnings': True,
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
    }
    
    for attempt in range(max_retries):
        try:
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                ydl.download([f'https://www.youtube.com/watch?v={youtube_id}'])
            return True
        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)  # Exponential backoff
            else:
                return False
    return False

# Download with rate limiting
if to_download:
    print(f"\nDownloading {len(to_download)} audio files...")
    for i, entry in enumerate(tqdm(to_download, desc="Downloading")):
        yt_id = entry['youtube_id']
        
        success = download_youtube_audio(yt_id, PSYLLABUS_AUDIO_DIR)
        
        if success:
            download_checkpoint['completed'].append(yt_id)
        else:
            download_checkpoint['failed'].append(yt_id)
        
        # Save checkpoint periodically
        if (i + 1) % 10 == 0:
            with open(PSYLLABUS_CHECKPOINT_FILE, 'w') as f:
                json.dump(download_checkpoint, f)
        
        # Rate limiting to avoid YouTube blocks
        time.sleep(random.uniform(1, 3))
    
    # Final checkpoint save
    with open(PSYLLABUS_CHECKPOINT_FILE, 'w') as f:
        json.dump(download_checkpoint, f)

# Count downloaded audio
downloaded_audio = list(PSYLLABUS_AUDIO_DIR.glob('*.wav'))
print(f"\nTotal PSyllabus audio files: {len(downloaded_audio)}")

In [ ]:
# Cell 21: Extract MuQ embeddings for PSyllabus
PSYLLABUS_MUQ_CACHE = PSYLLABUS_DIR / 'muq_cache' / 'last_hidden_state'
PSYLLABUS_MUQ_CACHE.mkdir(parents=True, exist_ok=True)

# Get list of available audio keys
psyllabus_audio_keys = [p.stem for p in PSYLLABUS_AUDIO_DIR.glob('*.wav')]
print(f"PSyllabus audio files: {len(psyllabus_audio_keys)}")

# Check cached embeddings
cached_psyllabus = {p.stem for p in PSYLLABUS_MUQ_CACHE.glob('*.pt')}
missing_psyllabus = [k for k in psyllabus_audio_keys if k not in cached_psyllabus]
print(f"Cached: {len(cached_psyllabus)}, Missing: {len(missing_psyllabus)}")

# Extract missing embeddings
if missing_psyllabus:
    print(f"\nExtracting {len(missing_psyllabus)} MuQ embeddings for PSyllabus...")
    extract_muq_embeddings(PSYLLABUS_AUDIO_DIR, PSYLLABUS_MUQ_CACHE, missing_psyllabus,
                           layer_start=None, layer_end=None)

In [ ]:
# Cell 22: Run X3 - PSyllabus Difficulty Correlation
exp_id = 'X3_psyllabus_difficulty'

if should_run_experiment(exp_id, CHECKPOINT_ROOT, RESULTS_DIR, GDRIVE_RESULTS, COMPLETED_CACHE):
    print(f"\n{'='*70}")
    print(f"X3: PSYLLABUS DIFFICULTY CORRELATION")
    print(f"{'='*70}")
    
    # Check if we have any samples to process
    if not sampled_entries:
        print("ERROR: No PSyllabus entries were parsed successfully.")
        print("Please check the data parsing in Cell 18.")
        print("Expected fields: youtube_link (for video ID), ps_rating or syllabus (for difficulty)")
        raise ValueError("No PSyllabus entries available for processing")
    
    # Load best D8 model (fold 0 for inference)
    d8_ckpt = CHECKPOINT_ROOT / 'D8_muq_stats' / 'fold0_best.ckpt'
    if not d8_ckpt.exists():
        raise FileNotFoundError(f"D8 checkpoint not found: {d8_ckpt}")
    
    model = MuQStatsModel.load_from_checkpoint(d8_ckpt)
    model = model.to('cuda').eval()
    
    # Create mapping from youtube_id to difficulty
    yt_to_difficulty = {e['youtube_id']: e['difficulty'] for e in sampled_entries}
    
    # Collect predictions
    predictions = []
    difficulties = []
    
    for key in tqdm(psyllabus_audio_keys, desc="PSyllabus inference"):
        if key not in yt_to_difficulty:
            continue
        
        emb_path = PSYLLABUS_MUQ_CACHE / f"{key}.pt"
        if not emb_path.exists():
            continue
        
        try:
            with torch.no_grad():
                emb = torch.load(emb_path, weights_only=True).unsqueeze(0).cuda()
                mask = torch.ones(1, emb.shape[1], dtype=torch.bool).cuda()
                pred = model(emb, mask).cpu().numpy()[0]
            
            predictions.append(pred)
            difficulties.append(yt_to_difficulty[key])
        except Exception as e:
            print(f"Warning: Failed to process {key}: {e}")
    
    predictions = np.array(predictions)
    difficulties = np.array(difficulties)
    
    print(f"\nProcessed {len(predictions)} PSyllabus samples")
    
    # Check if we have enough samples
    if len(predictions) == 0:
        print("WARNING: No samples were processed successfully.")
        print("This could be because:")
        print("  1. No audio files were downloaded (check Cell 20)")
        print("  2. No MuQ embeddings were extracted (check Cell 21)")
        print("  3. YouTube IDs don't match between parsed entries and downloaded files")
        
        x3_results = {
            'experiment_id': exp_id,
            'n_samples': 0,
            'error': 'No samples processed - check data pipeline',
            'overall_correlation': {'spearman_r': None, 'p_value': None},
            'dimension_correlations': {},
            'per_difficulty_stats': {},
        }
    else:
        # Compute correlations
        dimension_correlations = {}
        for i, dim in enumerate(PERCEPIANO_DIMENSIONS):
            corr, pval = stats.spearmanr(difficulties, predictions[:, i])
            dimension_correlations[dim] = {
                'spearman_r': float(corr),
                'p_value': float(pval),
                'significant': pval < 0.05,
            }
        
        # Overall mean prediction correlation with difficulty
        mean_pred = predictions.mean(axis=1)
        overall_corr, overall_pval = stats.spearmanr(difficulties, mean_pred)
        
        # Per-difficulty statistics
        per_difficulty_stats = {}
        for diff in range(1, 12):
            mask = difficulties == diff
            if mask.sum() > 0:
                per_difficulty_stats[diff] = {
                    'count': int(mask.sum()),
                    'mean_overall': float(predictions[mask].mean()),
                    'std_overall': float(predictions[mask].std()),
                }
        
        x3_results = {
            'experiment_id': exp_id,
            'n_samples': len(predictions),
            'overall_correlation': {
                'spearman_r': float(overall_corr),
                'p_value': float(overall_pval),
            },
            'dimension_correlations': dimension_correlations,
            'per_difficulty_stats': per_difficulty_stats,
        }
        
        # Print summary
        print(f"\n" + "="*50)
        print(f"X3 PSYLLABUS RESULTS:")
        print(f"="*50)
        print(f"Overall mean prediction vs difficulty:")
        print(f"  Spearman r = {overall_corr:.4f} (p = {overall_pval:.4e})")
        print(f"\nTop correlated dimensions:")
        sorted_dims = sorted(dimension_correlations.items(), key=lambda x: abs(x[1]['spearman_r']), reverse=True)
        for dim, data in sorted_dims[:5]:
            sig = '*' if data['significant'] else ''
            print(f"  {dim}: r = {data['spearman_r']:.4f}{sig}")
    
    # Save results
    with open(RESULTS_DIR / f'{exp_id}.json', 'w') as f:
        json.dump(x3_results, f, indent=2, default=numpy_serializer)
    
    ALL_RESULTS[exp_id] = x3_results
    
    # Sync to GDrive
    sync_experiment_to_gdrive(exp_id, x3_results, RESULTS_DIR, CHECKPOINT_ROOT, GDRIVE_RESULTS, ALL_RESULTS)
    
    del model
    torch.cuda.empty_cache()
else:
    print(f"\nSKIP {exp_id}: already complete")

---
## Part 3: Performer-Fold Evaluation (Optional)

**Goal:** Test if the model generalizes to unseen performers.

**Method:** Instead of piece-based folds, use performer-based folds. This tests whether the model learned general performance features vs. just memorizing performer styles.

**Expected outcome:** If R2 drops significantly, model is capturing performer-specific features. If R2 holds, model captures generalizable performance qualities.

In [ ]:
# Cell 24: Create performer-based folds
# First, analyze the performer distribution in PercePiano

# Parse performer information from keys
# PercePiano keys format: composer_piece_performer (e.g., "Bach_WTC1_Fugue1_performer123")
performer_to_keys = {}
key_to_performer = {}

for key in ALL_KEYS:
    # Extract performer ID (usually last part after underscore)
    parts = key.split('_')
    # The last numeric part is typically the performer ID
    performer_id = None
    for part in reversed(parts):
        if part.isdigit() or part.startswith('p') or 'performer' in part.lower():
            performer_id = part
            break
    
    if performer_id is None:
        # Use last part as performer ID
        performer_id = parts[-1] if len(parts) > 1 else 'unknown'
    
    if performer_id not in performer_to_keys:
        performer_to_keys[performer_id] = []
    performer_to_keys[performer_id].append(key)
    key_to_performer[key] = performer_id

print(f"Unique performers: {len(performer_to_keys)}")
print(f"\nPerformer distribution (top 10):")
for perf, keys in sorted(performer_to_keys.items(), key=lambda x: len(x[1]), reverse=True)[:10]:
    print(f"  {perf}: {len(keys)} samples")

In [ ]:
# Cell 25: Create performer-stratified folds
from sklearn.model_selection import KFold

# Sort performers by number of samples (for reproducibility)
performers = sorted(performer_to_keys.keys(), key=lambda x: len(performer_to_keys[x]), reverse=True)

# Create 4-fold split by performers
kf = KFold(n_splits=4, shuffle=True, random_state=42)

PERFORMER_FOLD_ASSIGNMENTS = {f'fold_{i}': [] for i in range(4)}

for fold_idx, (train_perfs, val_perfs) in enumerate(kf.split(performers)):
    val_performer_ids = [performers[i] for i in val_perfs]
    for perf_id in val_performer_ids:
        PERFORMER_FOLD_ASSIGNMENTS[f'fold_{fold_idx}'].extend(performer_to_keys[perf_id])

print("Performer-based fold assignments:")
for fold_id in range(4):
    n_samples = len(PERFORMER_FOLD_ASSIGNMENTS[f'fold_{fold_id}'])
    print(f"  Fold {fold_id}: {n_samples} samples")

# Check overlap with original folds
print("\nOverlap with original piece-based folds:")
for fold_id in range(4):
    original = set(FOLD_ASSIGNMENTS.get(f'fold_{fold_id}', []))
    performer_based = set(PERFORMER_FOLD_ASSIGNMENTS[f'fold_{fold_id}'])
    overlap = len(original & performer_based)
    print(f"  Fold {fold_id}: {overlap}/{len(original)} overlap ({100*overlap/len(original):.1f}%)")

In [ ]:
# Cell 26: P1 - Performer-Fold MuQ Experiment
exp_id = 'P1_performer_fold_muq'

if should_run_experiment(exp_id, CHECKPOINT_ROOT, RESULTS_DIR, GDRIVE_RESULTS, COMPLETED_CACHE):
    print(f"\n{'='*70}")
    print(f"P1: PERFORMER-FOLD MUQ EXPERIMENT")
    print(f"Description: MuQ with performer-based cross-validation")
    print(f"{'='*70}")
    
    # Use the same MuQ embeddings and model config as D8
    P1_RESULTS = run_4fold_mert_experiment(
        exp_id=exp_id,
        description='MuQ with stats pooling - performer-based folds',
        model_factory=make_muq_stats_model,
        mert_cache_dir=MUQ_CACHE_DIR,
        labels=LABELS,
        fold_assignments=PERFORMER_FOLD_ASSIGNMENTS,
        config=MUQ_STATS_CONFIG,
        checkpoint_root=CHECKPOINT_ROOT,
        results_dir=RESULTS_DIR,
        log_dir=LOG_DIR,
    )
    
    ALL_RESULTS[exp_id] = P1_RESULTS
    sync_experiment_to_gdrive(exp_id, P1_RESULTS, RESULTS_DIR, CHECKPOINT_ROOT, GDRIVE_RESULTS, ALL_RESULTS)
    
    # Compare with D8 (piece-based folds)
    print(f"\n" + "="*50)
    print(f"COMPARISON: Piece-fold vs Performer-fold")
    print(f"="*50)
    
    d8_r2 = ALL_RESULTS.get('D8_muq_stats', {}).get('summary', {}).get('avg_r2', 'N/A')
    p1_r2 = P1_RESULTS['summary']['avg_r2']
    
    print(f"D8 (piece-fold): R2 = {d8_r2}")
    print(f"P1 (performer-fold): R2 = {p1_r2:.4f}")
    
    if isinstance(d8_r2, float):
        diff = d8_r2 - p1_r2
        print(f"\nDifference: {diff:.4f} ({100*diff/d8_r2:.1f}% drop)")
        
        if diff > 0.05:
            print("Interpretation: Significant drop suggests model learns performer-specific features.")
        else:
            print("Interpretation: Small drop suggests model learns generalizable performance features.")
else:
    print(f"\nSKIP {exp_id}: already complete")

---
## Part 5: Multi-Model Performer-Fold Comparison (Nice-to-Have)

**Goal:** Compare how different models handle performer generalization.

**Hypothesis:** 
- If MuQ shows a big drop on performer-fold but MERT or symbolic doesn't, it suggests MuQ captures more performer-specific features
- If symbolic drops less, it suggests symbolic features are more generalizable (piece characteristics vs performance style)

**Models to compare:**
- P1: MuQ (already run above)
- P2: MERT (D1a_stats equivalent with performer folds)
- P3: Symbolic baseline (if available)

In [ ]:
# Cell: Download MERT embeddings for performer-fold experiments
from audio_experiments.extractors import extract_mert_for_layer_range
from audio_experiments.models import StatsPoolingModel

# MERT cache for layers 7-12 (best performing range from prior experiments)
MERT_CACHE_DIR = DATA_ROOT / 'mert_cache' / 'L7-12'
MERT_CACHE_DIR.mkdir(parents=True, exist_ok=True)

GDRIVE_MERT_CACHE = 'gdrive:crescendai_data/audio_baseline/mert_embeddings/L7-12'

# Try to download cached MERT embeddings
result = subprocess.run(['rclone', 'lsf', GDRIVE_MERT_CACHE], capture_output=True, text=True)
if result.returncode == 0 and result.stdout.strip():
    print("Downloading cached MERT embeddings (L7-12)...")
    run_rclone(['rclone', 'copy', GDRIVE_MERT_CACHE, str(MERT_CACHE_DIR), '--progress'],
               "Downloading MERT embeddings")
else:
    print("No cached MERT embeddings found.")

# Check what we have
cached_mert = {p.stem for p in MERT_CACHE_DIR.glob('*.pt')}
missing_mert = [k for k in ALL_KEYS if k not in cached_mert]
print(f"MERT L7-12 cached: {len(cached_mert)}, Missing: {len(missing_mert)}")

# Extract missing embeddings if needed
if missing_mert:
    print(f"\nExtracting {len(missing_mert)} MERT embeddings...")
    extract_mert_for_layer_range(AUDIO_DIR, MERT_CACHE_DIR, missing_mert, 
                                  layer_start=7, layer_end=12)

In [ ]:
# Cell: P2 - MERT Performer-Fold Experiment
exp_id = 'P2_performer_fold_mert'

# MERT stats pooling config (matching D1a_stats from prior experiments)
MERT_STATS_CONFIG = {
    **BASE_CONFIG,
    'input_dim': 1024,
    'hidden_dim': 512,
    'dropout': 0.2,
    'learning_rate': 1e-4,
    'weight_decay': 1e-5,
    'pooling_stats': 'mean_std',
}

def make_mert_stats_model(cfg):
    return StatsPoolingModel(
        input_dim=cfg['input_dim'],
        hidden_dim=cfg['hidden_dim'],
        dropout=cfg['dropout'],
        learning_rate=cfg['learning_rate'],
        weight_decay=cfg['weight_decay'],
        pooling_stats=cfg['pooling_stats'],
        max_epochs=cfg['max_epochs'],
    )

if should_run_experiment(exp_id, CHECKPOINT_ROOT, RESULTS_DIR, GDRIVE_RESULTS, COMPLETED_CACHE):
    print(f"\n{'='*70}")
    print(f"P2: MERT PERFORMER-FOLD EXPERIMENT")
    print(f"Description: MERT with stats pooling - performer-based folds")
    print(f"{'='*70}")
    
    P2_RESULTS = run_4fold_mert_experiment(
        exp_id=exp_id,
        description='MERT with stats pooling - performer-based folds',
        model_factory=make_mert_stats_model,
        mert_cache_dir=MERT_CACHE_DIR,
        labels=LABELS,
        fold_assignments=PERFORMER_FOLD_ASSIGNMENTS,
        config=MERT_STATS_CONFIG,
        checkpoint_root=CHECKPOINT_ROOT,
        results_dir=RESULTS_DIR,
        log_dir=LOG_DIR,
    )
    
    ALL_RESULTS[exp_id] = P2_RESULTS
    sync_experiment_to_gdrive(exp_id, P2_RESULTS, RESULTS_DIR, CHECKPOINT_ROOT, GDRIVE_RESULTS, ALL_RESULTS)
else:
    print(f"\nSKIP {exp_id}: already complete")

In [ ]:
# Cell: Multi-Model Performer-Fold Comparison Summary
print("\n" + "="*70)
print("MULTI-MODEL PERFORMER-FOLD COMPARISON")
print("="*70)

# Reference piece-fold results (from prior experiments)
PIECE_FOLD_BASELINES = {
    'D8_muq_stats': 0.560,  # Will be updated with complete 4-fold result
    'D1a_stats': 0.466,     # MERT piece-fold from definitive experiments
}

# Load actual D8 result if available
if 'D8_muq_stats' in ALL_RESULTS:
    PIECE_FOLD_BASELINES['D8_muq_stats'] = ALL_RESULTS['D8_muq_stats']['summary']['avg_r2']

comparison_data = []

# P1: MuQ performer-fold
if 'P1_performer_fold_muq' in ALL_RESULTS:
    p1_r2 = ALL_RESULTS['P1_performer_fold_muq']['summary']['avg_r2']
    base_r2 = PIECE_FOLD_BASELINES.get('D8_muq_stats', 0)
    drop = base_r2 - p1_r2 if base_r2 > 0 else 0
    drop_pct = 100 * drop / base_r2 if base_r2 > 0 else 0
    comparison_data.append({
        'model': 'MuQ (P1)',
        'piece_fold': base_r2,
        'performer_fold': p1_r2,
        'drop': drop,
        'drop_pct': drop_pct,
    })
    print(f"\nMuQ:")
    print(f"  Piece-fold R2: {base_r2:.4f}")
    print(f"  Performer-fold R2: {p1_r2:.4f}")
    print(f"  Drop: {drop:.4f} ({drop_pct:.1f}%)")

# P2: MERT performer-fold
if 'P2_performer_fold_mert' in ALL_RESULTS:
    p2_r2 = ALL_RESULTS['P2_performer_fold_mert']['summary']['avg_r2']
    base_r2 = PIECE_FOLD_BASELINES.get('D1a_stats', 0)
    drop = base_r2 - p2_r2 if base_r2 > 0 else 0
    drop_pct = 100 * drop / base_r2 if base_r2 > 0 else 0
    comparison_data.append({
        'model': 'MERT (P2)',
        'piece_fold': base_r2,
        'performer_fold': p2_r2,
        'drop': drop,
        'drop_pct': drop_pct,
    })
    print(f"\nMERT:")
    print(f"  Piece-fold R2: {base_r2:.4f}")
    print(f"  Performer-fold R2: {p2_r2:.4f}")
    print(f"  Drop: {drop:.4f} ({drop_pct:.1f}%)")

# Analysis
print(f"\n" + "-"*50)
print("INTERPRETATION:")
if comparison_data:
    muq_drop = next((d['drop_pct'] for d in comparison_data if 'MuQ' in d['model']), None)
    mert_drop = next((d['drop_pct'] for d in comparison_data if 'MERT' in d['model']), None)
    
    if muq_drop is not None and mert_drop is not None:
        if abs(muq_drop - mert_drop) < 5:
            print("Similar drop across models - performer variation affects both equally")
            print("Suggests inherent dataset characteristics, not model-specific bias")
        elif muq_drop > mert_drop:
            print(f"MuQ drops more ({muq_drop:.1f}% vs {mert_drop:.1f}%)")
            print("MuQ may capture more performer-specific features (timbre, touch)")
        else:
            print(f"MERT drops more ({mert_drop:.1f}% vs {muq_drop:.1f}%)")
            print("MERT may capture more performer-specific features")
else:
    print("Run P1 and P2 experiments first for comparison")

---
## Part 6: Soundfont Augmentation (Nice-to-Have)

**Goal:** Test if augmenting training data with multiple Pianoteq soundfonts improves generalization.

**Hypothesis:**
- Since PercePiano is rendered MIDI, the same "performance" can be rendered with different piano sounds
- This should improve timbre-invariance for dimensions like timing, articulation, tempo
- Timbre dimensions (brightness, depth) will legitimately vary across soundfonts

**Approach:**
1. Download original MIDI files
2. Re-render with 3-4 different Pianoteq presets (Steinway D, Bosendorfer, Yamaha C7, etc.)
3. Extract MuQ embeddings for augmented audio
4. Train with augmented data (labels remain the same for all soundfont versions)
5. Compare R2 on held-out soundfont

**Note:** This requires Pianoteq CLI to be installed. If not available, this section will be skipped.

In [ ]:
# Cell: Check Pianoteq availability and setup soundfont augmentation
import shutil

# Check if Pianoteq CLI is available
PIANOTEQ_PATH = shutil.which('pianoteq') or shutil.which('Pianoteq')
PIANOTEQ_AVAILABLE = PIANOTEQ_PATH is not None

# Alternative: check for common installation paths
if not PIANOTEQ_AVAILABLE:
    common_paths = [
        '/Applications/Pianoteq 8/Pianoteq 8.app/Contents/MacOS/Pianoteq 8',
        '/opt/pianoteq/pianoteq',
        os.path.expanduser('~/Pianoteq 8/Pianoteq 8'),
    ]
    for path in common_paths:
        if os.path.exists(path):
            PIANOTEQ_PATH = path
            PIANOTEQ_AVAILABLE = True
            break

print(f"Pianoteq available: {PIANOTEQ_AVAILABLE}")
if PIANOTEQ_AVAILABLE:
    print(f"Pianoteq path: {PIANOTEQ_PATH}")

# Define soundfont presets to use for augmentation
SOUNDFONT_PRESETS = [
    'Steinway Model D',      # Bright concert grand (original)
    'NY Steinway Model D',   # Warmer variant
    'Bosendorfer 280VC',     # Rich Viennese sound
    'Yamaha C7',             # Bright Japanese sound
]

# Directories
MIDI_DIR = DATA_ROOT / 'midi'
AUGMENTED_AUDIO_DIR = DATA_ROOT / 'audio_augmented'

MIDI_DIR.mkdir(parents=True, exist_ok=True)
AUGMENTED_AUDIO_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
# Cell: Download MIDI files and render with multiple soundfonts
GDRIVE_MIDI = 'gdrive:crescendai_data/percepiano_midi'

if PIANOTEQ_AVAILABLE:
    # Download MIDI files
    print("Downloading MIDI files...")
    result = subprocess.run(['rclone', 'lsf', GDRIVE_MIDI], capture_output=True, text=True)
    
    if result.returncode == 0 and result.stdout.strip():
        run_rclone(['rclone', 'copy', GDRIVE_MIDI, str(MIDI_DIR), '--progress'],
                   "Downloading MIDI files")
        midi_files = list(MIDI_DIR.glob('*.mid')) + list(MIDI_DIR.glob('*.midi'))
        print(f"Downloaded {len(midi_files)} MIDI files")
    else:
        print("Warning: MIDI files not found on GDrive")
        print("Soundfont augmentation requires original MIDI files")
        midi_files = []
    
    # Render function
    def render_midi_with_pianoteq(midi_path: Path, output_path: Path, preset: str) -> bool:
        """Render MIDI file with Pianoteq using specified preset."""
        cmd = [
            PIANOTEQ_PATH,
            '--headless',
            '--preset', preset,
            '--midi', str(midi_path),
            '--wav', str(output_path),
            '--rate', '24000',  # Match MuQ sample rate
        ]
        try:
            result = subprocess.run(cmd, capture_output=True, text=True, timeout=120)
            return result.returncode == 0 and output_path.exists()
        except subprocess.TimeoutExpired:
            return False
        except Exception:
            return False
    
    # Render with each preset
    if midi_files:
        augmented_data = {}
        
        for preset_idx, preset in enumerate(SOUNDFONT_PRESETS[1:], 1):  # Skip first (original)
            preset_short = preset.replace(' ', '_').lower()[:10]
            preset_dir = AUGMENTED_AUDIO_DIR / preset_short
            preset_dir.mkdir(parents=True, exist_ok=True)
            
            print(f"\nRendering with {preset} (preset {preset_idx}/{len(SOUNDFONT_PRESETS)-1})...")
            
            rendered_count = 0
            for midi_path in tqdm(midi_files, desc=f"Rendering {preset_short}"):
                key = midi_path.stem
                output_path = preset_dir / f"{key}.wav"
                
                if output_path.exists():
                    rendered_count += 1
                    continue
                
                if render_midi_with_pianoteq(midi_path, output_path, preset):
                    rendered_count += 1
            
            augmented_data[preset_short] = {
                'preset': preset,
                'dir': preset_dir,
                'count': rendered_count,
            }
            print(f"  Rendered: {rendered_count}/{len(midi_files)}")
else:
    print("\nSkipping soundfont augmentation: Pianoteq not available")
    print("To enable, install Pianoteq and ensure CLI is accessible")

In [ ]:
# Cell: S1 - Soundfont Augmentation Training Experiment
exp_id = 'S1_soundfont_augmented'

if PIANOTEQ_AVAILABLE and 'augmented_data' in dir() and augmented_data:
    if should_run_experiment(exp_id, CHECKPOINT_ROOT, RESULTS_DIR, GDRIVE_RESULTS, COMPLETED_CACHE):
        print(f"\n{'='*70}")
        print(f"S1: SOUNDFONT AUGMENTATION EXPERIMENT")
        print(f"{'='*70}")
        
        # Extract MuQ embeddings for augmented audio
        AUGMENTED_MUQ_CACHE = DATA_ROOT / 'muq_cache' / 'augmented'
        
        for preset_short, data in augmented_data.items():
            preset_cache = AUGMENTED_MUQ_CACHE / preset_short
            preset_cache.mkdir(parents=True, exist_ok=True)
            
            audio_files = list(data['dir'].glob('*.wav'))
            keys = [f.stem for f in audio_files]
            
            cached = {p.stem for p in preset_cache.glob('*.pt')}
            missing = [k for k in keys if k not in cached]
            
            if missing:
                print(f"\nExtracting MuQ embeddings for {preset_short} ({len(missing)} files)...")
                extract_muq_embeddings(data['dir'], preset_cache, missing,
                                      layer_start=None, layer_end=None)
        
        # Create augmented dataset with all soundfonts
        # Labels remain the same (same MIDI performance, different timbre)
        AUGMENTED_LABELS = {}
        AUGMENTED_FOLD_ASSIGNMENTS = {f'fold_{i}': [] for i in range(4)}
        
        # Original data
        for key, label in LABELS.items():
            AUGMENTED_LABELS[f'original_{key}'] = label
            fold = FOLD_BY_KEY.get(key)
            if fold is not None:
                AUGMENTED_FOLD_ASSIGNMENTS[f'fold_{fold}'].append(f'original_{key}')
        
        # Augmented data (same fold as original)
        for preset_short in augmented_data.keys():
            preset_cache = AUGMENTED_MUQ_CACHE / preset_short
            for key in LABELS.keys():
                if (preset_cache / f'{key}.pt').exists():
                    aug_key = f'{preset_short}_{key}'
                    AUGMENTED_LABELS[aug_key] = LABELS[key]  # Same label
                    fold = FOLD_BY_KEY.get(key)
                    if fold is not None:
                        AUGMENTED_FOLD_ASSIGNMENTS[f'fold_{fold}'].append(aug_key)
        
        print(f"\nAugmented dataset:")
        print(f"  Original samples: {len(LABELS)}")
        print(f"  Total samples: {len(AUGMENTED_LABELS)}")
        print(f"  Augmentation factor: {len(AUGMENTED_LABELS) / len(LABELS):.1f}x")
        
        # Custom dataset that loads from multiple cache directories
        class AugmentedMERTDataset:
            def __init__(self, cache_dirs, labels, keys, max_frames=1000):
                self.cache_dirs = cache_dirs
                self.labels = labels
                self.keys = keys
                self.max_frames = max_frames
            
            def __len__(self):
                return len(self.keys)
            
            def __getitem__(self, idx):
                key = self.keys[idx]
                
                # Determine which cache dir to use
                if key.startswith('original_'):
                    actual_key = key[9:]  # Remove 'original_' prefix
                    cache_dir = self.cache_dirs['original']
                else:
                    parts = key.split('_', 1)
                    preset_short = parts[0]
                    actual_key = parts[1]
                    cache_dir = self.cache_dirs.get(preset_short, self.cache_dirs['original'])
                
                emb_path = cache_dir / f'{actual_key}.pt'
                if not emb_path.exists():
                    raise FileNotFoundError(f"Missing embedding: {emb_path}")
                
                emb = torch.load(emb_path, weights_only=True)
                if emb.shape[0] > self.max_frames:
                    emb = emb[:self.max_frames]
                
                label = torch.tensor(self.labels[key], dtype=torch.float32)
                return {'embeddings': emb, 'labels': label, 'key': key}
        
        # Build cache dirs mapping
        cache_dirs = {'original': MUQ_CACHE_DIR}
        for preset_short in augmented_data.keys():
            cache_dirs[preset_short] = AUGMENTED_MUQ_CACHE / preset_short
        
        # Run experiment (simplified - single fold for proof of concept)
        fold = 0
        train_keys = [k for k in AUGMENTED_LABELS.keys() 
                     if k not in AUGMENTED_FOLD_ASSIGNMENTS[f'fold_{fold}']]
        val_keys = AUGMENTED_FOLD_ASSIGNMENTS[f'fold_{fold}']
        
        # Only validate on original data (not augmented)
        val_keys_original = [k for k in val_keys if k.startswith('original_')]
        
        print(f"\nFold 0: {len(train_keys)} train (augmented), {len(val_keys_original)} val (original only)")
        
        # Note: Full training would use run_4fold_mert_experiment with custom dataset
        # This is a simplified proof of concept
        print("\nNote: Full soundfont augmentation training is compute-intensive.")
        print("Running single fold as proof of concept...")
        
        # TODO: Implement full 4-fold training with augmented data
        # For now, just document the approach
        
        s1_results = {
            'experiment_id': exp_id,
            'status': 'proof_of_concept',
            'augmentation_factor': len(AUGMENTED_LABELS) / len(LABELS),
            'soundfonts_used': list(augmented_data.keys()),
            'original_samples': len(LABELS),
            'augmented_samples': len(AUGMENTED_LABELS),
            'note': 'Full training requires significant compute. Framework ready for execution.',
        }
        
        ALL_RESULTS[exp_id] = s1_results
        
        with open(RESULTS_DIR / f'{exp_id}.json', 'w') as f:
            json.dump(s1_results, f, indent=2)
        
        sync_experiment_to_gdrive(exp_id, s1_results, RESULTS_DIR, CHECKPOINT_ROOT, GDRIVE_RESULTS, ALL_RESULTS)
else:
    print("\nSkipping S1: Pianoteq not available or no augmented data rendered")

---
## Part 4: Final Summary

Generate paper-ready summary of all experiments.

In [ ]:
# Cell 28: Load all results from GDrive
print("Loading all experiment results...")

# Load D8 results
result = subprocess.run(['rclone', 'cat', f'{GDRIVE_D8_RESULTS}/D8_muq_stats.json'],
                       capture_output=True, text=True)
if result.returncode == 0:
    ALL_RESULTS['D8_muq_stats'] = json.loads(result.stdout)

# Load new experiment results
for exp_id in ['X3_psyllabus_difficulty', 'P1_performer_fold_muq', 
               'P2_performer_fold_mert', 'S1_soundfont_augmented']:
    result = subprocess.run(['rclone', 'cat', f'{GDRIVE_RESULTS}/{exp_id}.json'],
                           capture_output=True, text=True)
    if result.returncode == 0:
        ALL_RESULTS[exp_id] = json.loads(result.stdout)

print(f"Loaded {len(ALL_RESULTS)} experiment results")

In [ ]:
# Cell 29: Generate Final Summary
print("\n" + "="*70)
print("PAPER-READY EXPERIMENTS: FINAL SUMMARY")
print("="*70)

# D8 Results
if 'D8_muq_stats' in ALL_RESULTS:
    d8 = ALL_RESULTS['D8_muq_stats']
    summary = d8.get('summary', {})
    fold_results = d8.get('fold_results', {})
    
    print(f"\n1. D8_muq_stats (MuQ + Stats Pooling):")
    print(f"   Per-fold R2: {[f'{float(v):.4f}' for v in fold_results.values()]}")
    print(f"   Average R2: {summary.get('avg_r2', 'N/A'):.4f} +/- {summary.get('std_r2', 0):.4f}")
    print(f"   95% CI: [{summary.get('r2_ci_95', [0,0])[0]:.4f}, {summary.get('r2_ci_95', [0,0])[1]:.4f}]")
    print(f"   Note: {d8.get('note', 'Complete 4-fold CV')}")

# PSyllabus Results
if 'X3_psyllabus_difficulty' in ALL_RESULTS:
    x3 = ALL_RESULTS['X3_psyllabus_difficulty']
    overall = x3.get('overall_correlation', {})
    
    print(f"\n2. X3_psyllabus_difficulty (Cross-Dataset Validation):")
    print(f"   Samples: {x3.get('n_samples', 'N/A')}")
    print(f"   Overall correlation with difficulty: r = {overall.get('spearman_r', 'N/A'):.4f}")
    print(f"   p-value: {overall.get('p_value', 'N/A'):.4e}")
    
    # Top correlated dimensions
    dims = x3.get('dimension_correlations', {})
    sorted_dims = sorted(dims.items(), key=lambda x: abs(x[1].get('spearman_r', 0)), reverse=True)
    print(f"   Top correlated dimensions:")
    for dim, data in sorted_dims[:3]:
        sig = '*' if data.get('significant', False) else ''
        print(f"     - {dim}: r = {data.get('spearman_r', 0):.4f}{sig}")

# Performer-Fold Results
print(f"\n3. PERFORMER-FOLD COMPARISON:")
if 'P1_performer_fold_muq' in ALL_RESULTS:
    p1 = ALL_RESULTS['P1_performer_fold_muq']
    p1_summary = p1.get('summary', {})
    print(f"   P1 (MuQ): R2 = {p1_summary.get('avg_r2', 'N/A'):.4f}")
    
if 'P2_performer_fold_mert' in ALL_RESULTS:
    p2 = ALL_RESULTS['P2_performer_fold_mert']
    p2_summary = p2.get('summary', {})
    print(f"   P2 (MERT): R2 = {p2_summary.get('avg_r2', 'N/A'):.4f}")

# Compare drops
if 'D8_muq_stats' in ALL_RESULTS and 'P1_performer_fold_muq' in ALL_RESULTS:
    d8_r2 = ALL_RESULTS['D8_muq_stats'].get('summary', {}).get('avg_r2', 0)
    p1_r2 = ALL_RESULTS['P1_performer_fold_muq'].get('summary', {}).get('avg_r2', 0)
    muq_drop = d8_r2 - p1_r2
    print(f"   MuQ drop (piece->performer): {muq_drop:.4f} ({100*muq_drop/d8_r2:.1f}%)")
    
if 'P2_performer_fold_mert' in ALL_RESULTS:
    mert_piece_r2 = 0.466  # D1a_stats from definitive experiments
    p2_r2 = ALL_RESULTS['P2_performer_fold_mert'].get('summary', {}).get('avg_r2', 0)
    mert_drop = mert_piece_r2 - p2_r2
    print(f"   MERT drop (piece->performer): {mert_drop:.4f} ({100*mert_drop/mert_piece_r2:.1f}%)")

# Soundfont Augmentation Results
if 'S1_soundfont_augmented' in ALL_RESULTS:
    s1 = ALL_RESULTS['S1_soundfont_augmented']
    print(f"\n4. S1_soundfont_augmented:")
    print(f"   Status: {s1.get('status', 'N/A')}")
    print(f"   Augmentation factor: {s1.get('augmentation_factor', 'N/A'):.1f}x")
    print(f"   Soundfonts: {s1.get('soundfonts_used', [])}")

print("\n" + "="*70)
print("RECOMMENDATIONS FOR PAPER:")
print("="*70)
print("1. Use D8_muq_stats as headline result (complete 4-fold CV)")
print("2. Report PSyllabus correlation for external validation")
print("3. Include performer-fold comparison in limitations/analysis section")
print("4. If soundfont augmentation shows gains, report as data augmentation strategy")
print("="*70)

In [ ]:
# Cell 30: Save all results
# Save aggregate results
aggregate_file = RESULTS_DIR / 'paper_ready_all_results.json'
with open(aggregate_file, 'w') as f:
    json.dump(ALL_RESULTS, f, indent=2, default=numpy_serializer)

# Upload to GDrive
run_rclone(['rclone', 'copyto', str(aggregate_file), f'{GDRIVE_RESULTS}/paper_ready_all_results.json'],
           "Uploading aggregate results")

print(f"\nAll results saved to {aggregate_file}")
print(f"Uploaded to {GDRIVE_RESULTS}/paper_ready_all_results.json")

In [ ]:
# Cell 31: Verification commands
print("\nVERIFICATION COMMANDS:")
print("="*70)
print("\n# Verify D8 completion:")
print(f"rclone cat {GDRIVE_D8_RESULTS}/D8_muq_stats.json | python3 -c \"import json,sys; d=json.load(sys.stdin); print(f'Folds: {{list(d.get(\\\"fold_results\\\", {{}}).keys())}}'); print(f'Avg R2: {{d.get(\\\"summary\\\", {{}}).get(\\\"avg_r2\\\", \\\"N/A\\\")}}')\"")

print("\n# Verify PSyllabus results:")
print(f"rclone cat {GDRIVE_RESULTS}/X3_psyllabus_difficulty.json | python3 -c \"import json,sys; d=json.load(sys.stdin); print(f'Samples: {{d.get(\\\"n_samples\\\", \\\"N/A\\\")}}'); print(f'Correlation: {{d.get(\\\"overall_correlation\\\", {{}}).get(\\\"spearman_r\\\", \\\"N/A\\\")}}')\"")

print("\n# Verify performer-fold results:")
print(f"rclone cat {GDRIVE_RESULTS}/P1_performer_fold_muq.json | python3 -c \"import json,sys; d=json.load(sys.stdin); print(f'Avg R2: {{d.get(\\\"summary\\\", {{}}).get(\\\"avg_r2\\\", \\\"N/A\\\")}}')\"")